In [1]:
import json
import os
import numpy as np
import pandas as pd
import time

test = pd.read_json('test.json', typ='frame', encoding='utf-8')
tag_song = pd.read_json('tag_song_pair.json', typ='frame', orient= 'table', encoding='utf-8')
song_meta = pd.read_json('song_meta.json', typ='frame', encoding='utf-8')
tag_group = pd.read_json('tag_group.json', typ='frame', orient='table', encoding='utf-8')

In [2]:
genre_gn_all = pd.read_json('genre_gn_all.json', typ='series', encoding='utf-8')
genre_gn_all = pd.DataFrame(genre_gn_all, columns = ['gnr_name']).reset_index().rename(columns = {'index' : 'gnr_code'})

In [3]:
# 장르 리스트 key: 장르코드(GN~), value: 0
gen_list = {}
for i in range(254):
    gen_list[genre_gn_all.loc[i]['gnr_code']] = 0

In [4]:
# TAG 29160개 리스트로 저장
tag_list = tag_song['tag'].to_list()

In [9]:
# 태그가 부족할 때 태그-태그 매칭으로 태그 더 뽑기
def generate_tag(tList, lim):
    leftNum = lim-len(tList)     # 더 뽑아야하는 태그 수
    cand_tag = {}     # 후보 태그가 들어가는 딕셔너리
    res_list = []     # 최종적으로 반환할 태그 리스트
    for t in tList:     # tList에 있는 (소스태그들) 태그들을 모두 순회 -> t
        if t in tag_group['tag']:     # 만약 t가 29160개의 태그 중 하나라면
            for tt in tag_group[tag_group['tag']==t]['tag_group'].to_list()[0]:     # 태그 t와 매칭되는 모든 관련 태그들 (t - tt1, tt2, tt3....) 순회
                if tt not in tList:     # tt가 이미 소스태그로 주어진 태그(tList)가 아닐때만 추가
                    if tt not in list(cand_tag.keys()):     # tt가 아직 후보 태그로 선택되지 않았을 경우 (처음 발견된 태그인 경우)
                        cand_tag[tt] = 0     # 후보 태그로 추가하고 카운팅 횟수를 0으로 초기화 (처음 발견됐기 떄문에 0)
                    else: cand_tag[tt] += 1     # 이미 후보 태그 딕셔너리에 존재할 경우 카운트를 1번 추가
    for i in range(leftNum):     # 더 뽑아야하는 태그 수 만큼 반복
        if len(cand_tag.keys()) <= 0: return res_list     # 만약 후보 태그가 더 이상 존재하지 않을 경우 멈춤
        target = max(cand_tag.keys(), key=(lambda k: cand_tag[k]))     # 가장 많이 카운팅된 태그를 후보 태그에서 추출 (target)
        if target not in res_list: res_list.append(target)     # target이 최종 태그에 아직 추가되지 않았을 경우 최종 태그로 추가
        del cand_tag[target]     # target에 대해 확인이 끝났기 때문에 target을 후보 태그에서 삭제한다
    return res_list     # 최종태그를 반환한다. 후보 태그가 모두 소진되었을 경우 

# 플레이리스 idx의 id, tag list, song list 를 분해해서 반환
def give_tag_song(idx):
    id_ = test.loc[idx]['id'].astype(str)
    tags = test.loc[idx]['tags']
    songs = test.loc[idx]['songs']
    return id_, tags, songs

# 곡의 릴리즈 날짜를 확인하기 위해 복잡하게 써있는 날짜를 yyyymmdd 형태로 반환
def take_date(idx):
    target_date = test.loc[idx]['updt_date']
    year = target_date[:4]
    month = target_date[5:7]
    day = target_date[8:10]
    date = year+month+day
    return int(date)

# song(id) 가 idx 플레이리스트의 최종 수정 날짜 이전에 발매된 노래인지 확인
def valid_date(idx, id_):
    plylst_date = take_date(idx)
    song_date = song_meta[song_meta['id']==id_]['issue_date'].to_list()[0]
    if song_date < plylst_date: return True
    else: return False

# 충분히 뽑아야 할 태그 개수 반환
def setTagNumLimit(list_): return len(list_)+10
# 태그가 원하는 개수만큼 뽑혔는지 확인
def tagIsTen(list_, limit): return len(list_) == limit
# 곡이 원하는 개수만큼 뽑혔는지 확인
def songIsHund(list_): return len(list_) == 150


def resultCase_tag_song(idx):
    id_, tags, songs = give_tag_song(idx)     # 플레이리스 idx의 id, tag list, song list 
    tagNumLimit = setTagNumLimit(tags)     # 뽑으려는 태그 개수
    gen_list_copy = {}     # 장르 코드 - 카운트 매칭하는 딕셔너리
    tag_flag = 0 # 태그를 더 뽑아야하는지 확인하기 위한 변수
    
    # gen_list_copy[i]를 초기화. gen_list[i]를 그냥 0으로 바꿔주는게 나을 것 같음
    for i in genre_gn_all['gnr_code'].to_list(): 
        gen_list_copy[i] = gen_list[i]

    song_count = {}     # song id - 주어진 tag 안에서의 해당 곡의 출현 횟수 매칭 
    for s in songs:
        song_count[s] = 0
        
    for t in tags:     # 플레이리스트 idx에 존재하는 모든 태그에 대해 순회 -> t
        if t in tag_list:     # 만약 t가 29160개의 태그 안에 존재하는 태그라면
            for s in tag_song[tag_song['tag']==t]['song'].to_list()[0]:     # 태그 t에 매칭되는 곡들에 대해 출현횟수 카운팅
                if s in songs:
                    song_count[s] = song_count[s] + 1   


    tag_count = [0 for _ in range(29160)] # 태그-송 매칭에서 태그 별로 플레이리스트에 주어진 곡이 포함된 횟수 카운팅
    for i in range(29160):
        for s in tag_song.loc[i]['song']:
            if s in songs:
                tag_count[i] +=1
        
    # 태그 선택
    new_tags = []
    tag_count_copy = tag_count
    for i in range(tagNumLimit):     # 뽑고자 하는 태그 개수만큼 반복
        target = tag_count_copy.index(max(tag_count_copy))     # tag_count에서 가장 많이 카운팅된 태그 먼저 선택
        if tag_count_copy[target] == 0: break
        if tag_song.loc[target]["tag"] not in new_tags: new_tags.append(tag_song.loc[target]['tag'])
        tag_count_copy[target] = 0   
    del tag_count_copy 
    if tagIsTen(new_tags, tagNumLimit) is False: # 태그가 부족할 경우 generate_tag 함수를 통해 태그-태그 매칭에서 더 뽑기
        if len(new_tags) <= len(tags):
            new_tags = new_tags + tags
            new_tags = list(set(new_tags))
        new_tags = generate_tag(new_tags, tagNumLimit)
    if tagIsTen(new_tags, tagNumLimit) is False: # 그래도 태그가 부족할 경우 우선 tag_flag를 1로 설정한 뒤 곡 먼저 뽑기
        print("[ERROR] Need More Tag Flag")
        tag_flag = 1
    
    # 곡 선택
    new_songs = []
    new_song_dic = {}
    left_num = 150
    for t in new_tags:     # 위에서 뽑은 태그들에 대해 순회
        for s in tag_song[tag_song['tag']==t]['song'].to_list()[0]:
            # 태그-송 매칭에서 new_tag에 포함된 모든 곡들에 대해 카운팅 (이미 songs에 존재하는-플레이리스트에서 처음부터 주어진 곡들은 포함하지 않음)
            if s not in songs:
                if s not in new_song_dic:
                    new_song_dic[s] = 0     
                else: new_song_dic[s] = new_song_dic[s]+1
    while len(new_songs)<150:     # 곡 150개 뽑기 위해 150번 만큼 반복
        if len(new_song_dic.keys()) <= 0: break     # 후보 곡들이 모두 소진되었을 경우 멈춤
        target = max(new_song_dic.keys(), key=(lambda k: new_song_dic[k]))     # 태그-송 매칭에서 가장 많이 카운팅 된 순으로 뽑음
        if valid_date(idx, target) and target not in songs and target not in new_songs:
            new_songs.append(target)
            for g in song_meta[song_meta['id']==target]['song_gn_dtl_gnr_basket'].to_list()[0]:
                gen_list_copy[g] += 1
        del new_song_dic[target]
    if songIsHund(new_songs) is False: # 만약 곡이 부족할 경우 new_tag의 태그들 만이 아닌 전체 태그 29160개에 대해서 카운팅해줌
        new_song_dic = {}
        # 다시 보니 이 부분을 좀 수정해야 할 것 같아요
        for i in range(29160):
            for s in tag_song.loc[i]['song']:
                if s not in songs:
                    if s not in new_song_dic:
                        new_song_dic[s] = 0
                    else: new_song_dic[s] = new_song_dic[s]+1
        left_num = 150 - len(new_songs)
        for i in range(left_num): # 카운팅 후 더 뽑아야 하는 곡의 수 만큼 반복해서 많이 카운팅 된 순으로 뽑기
            if len(new_song_dic.keys()) <= 0: break
            target = max(new_song_dic.keys(), key=(lambda k: new_song_dic[k]))
            if valid_date(idx, target) and target not in songs and target not in new_songs:
                new_songs.append(target)
                for g in song_meta[song_meta['id']==target]['song_gn_dtl_gnr_basket'].to_list()[0]:
                    gen_list_copy[g] += 1
            del new_song_dic[target]
        if songIsHund(new_songs) is False: print("[ERROR] Need More Songs")

    # 이미 뽑힌 곡들에 대해 가장 많이 겹치는 장르를 먼저 골라 해당 장르의 곡에 우선순위를 둬서 거르는 코드입니다
    new_songs = list(set(new_songs))
    new_songs2 = []
    while True:
        if len(new_songs2)>=100 or len(gen_list_copy.keys())<=0: break
        tar_gen = max(gen_list_copy.keys(), key=(lambda k: gen_list_copy[k]))
        for i in new_songs:
            for j in song_meta[song_meta['id']==i]['song_gn_dtl_gnr_basket'].to_list()[0]:
                if j==tar_gen:
                    new_songs2.append(i)
                    new_songs.remove(i)
                    break
            if len(new_songs2)>=100: break
        del gen_list_copy[tar_gen]
    
    # 태그가 부족할 경우 추가로 뽑힌 곡들을 가지고 다시 위에서 했던 태그 뽑는 과정을 똑같이 반복합니다.
    if tag_flag == 1:
        print("Process Tag Flag")
        cand_tag = {}
        for i in range(29160):
            for s in tag_song.loc[i]['song']:
                if s in new_songs2[:10]:
                    if tag_song.loc[i]['tag'] not in new_tags:
                        if tag_song.loc[i]['tag'] not in list(cand_tag.keys()):
                            cand_tag[tag_song.loc[i]['tag']] = 0
                        else: cand_tag[tag_song.loc[i]['tag']] += 1
        leftLen = tagNumLimit-len(new_tags)
        for i in range(leftLen):
            if len(cand_tag.keys()) <= 0:  break
            target = max(cand_tag.keys(), key = (lambda k : cand_tag[k]))
            if target not in new_tags: new_tags.append(target)
            del cand_tag[target]
    if tagIsTen(new_tags, tagNumLimit) is False: print("[ERROR] Need More Tags")
    
    # 곡이 부족할 경우 위에서 했던 것과 같은 방법으로 곡을 더 뽑습니다
    new_song_dic = {}
    if len(new_songs2)<=100: # <100으로 고치는게 나을 것 같음
        left_num = 100-len(new_songs2)
        for t in new_tags:
            for s in tag_song[tag_song['tag']==t]['song'].to_list()[0]:
                if s not in songs:
                    if s not in new_song_dic:
                        new_song_dic[s] = 0
                    else: new_song_dic[s] = new_song_dic[s]+1
        while len(new_songs2)<100:
            if len(new_song_dic.keys()) <= 0: break
            target = max(new_song_dic.keys(), key=(lambda k: new_song_dic[k]))
            if valid_date(idx, target) and target not in songs:
                if target not in new_songs2: new_songs2.append(target)
            del new_song_dic[target]
    
    # 결과 리턴
    print("<Playlist {}>".format(idx))
    result_list = []
    result_list.append(id_)
    for t in new_tags:
        if t in tags: new_tags.remove(t)
    new_tags = new_tags[:10]
    result_list.append(new_tags)
    result_list.append(new_songs2)
    print("-- [Tags #]: " + str(len(new_tags)) + ", [Songs #]: " + str(len(new_songs2)))
    return result_list

In [10]:
re = resultCase_tag_song(1271)

[ERROR] Need More Tag Flag
Process Tag Flag
<Playlist 1271>
-- [Tags #]: 10, [Songs #]: 100


In [12]:
re[1]

['Pop', 'kpop', '가사', '가요', '가을', '가을감성', '가창력', '감동', '감미로운', '감성']

In [10]:
result_id = ['2240', '8083']
new = pd.DataFrame(data=result_id, columns=['id'])
new['tags'] = np.nan
new['songs'] = np.nan
new = new.to_numpy()
new[0, 1] = re[1]
new[0, 2] = re[2]
new[1, 1] = re2[1]
new[1, 2] = re2[2]
new2 = pd.DataFrame(data=new, columns=['id', 'tags', 'songs'])
new2.to_json('plylst_2240_8083.json')

In [ ]:
check = pd.read_json()